In [1]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


CUDA Available: True
GPU Name: NVIDIA GeForce GTX 1050


In [3]:
import pandas as pd

df = pd.read_csv('./../data/emails.csv')

In [4]:
print(df.shape)
df.head()

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [6]:
import email
email.message_from_string(df.iloc[0]['message'])

In [ ]:
import re
import unicodedata

def clean_email_line(text: str) -> str:
    if not isinstance(text, str):
        return ""

    # Normalize unicode
    text = unicodedata.normalize("NFKC", text)

    # Lowercase key header fields to remove
    header_keywords = [
        "return-path:", "received:", "x-originating-ip:", "x-from:",
        "x-to:", "mime-version:", "content-type:", "subject:", "to:",
        "from:", "cc:", "bcc:", "reply-to:", "sent:", "forwarded by",
        "http", "https", "www", "onebox.com"
    ]
    if any(text.lower().strip().strip('"').startswith(k) for k in header_keywords):
        return ""
    


    # Remove [image], [anything] tags
    text = re.sub(r"\[.*?\]", "", text)

    # Remove email addresses and phone numbers
    text = re.sub(r"[\w\.-]+@[\w\.-]+", "", text)
    text = re.sub(r"\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}", "", text)

    # Remove URLs
    text = re.sub(r"http[s]?://\S+|www\.\S+", "", text)

    # Remove HTML tags
    text = re.sub(r"<[^>]+>", "", text)

    # Remove long runs of punctuation, ???, ***
    text = re.sub(r"[\?\*]{3,}", "", text)
    text = re.sub(r"[-=*_]{3,}", "", text)
    text = re.sub(r"\.{4,}", ".", text)

    # Remove signature-style lines starting with "--"
    if text.strip().startswith("--"):
        return ""

    # Remove excessive whitespace
    text = re.sub(rr"\S+", " ", text).strip()

    # Remove isolated junk lines (too short or too weird)
    if len(text) < 20 or len(text.split()) < 4:
        return ""

    # Remove leading/trailing quotes
    text = re.sub(r"^[\"']+|[\"']+$", "", text)

    return text


In [ ]:
content_series = df['content'].dropna().astype(str)
content_series = content_series.apply(clean_email_line)
content_series = content_series[content_series.str.len() > 20]  # skip very short lines
content_series = content_series.sample(5000)

# Save to file
content_series.to_csv("./../data/accepted_emails.txt", index=False, header=False)


Cleaning and processing 2

In [8]:
# Convert to message objects from the message strings
messages = list(map(email.message_from_string, df['message']))

def get_text_from_email(msg):
  parts = []
  for part in msg.walk():
      if part.get_content_type() == 'text/plain':
          parts.append( part.get_payload() )
  text = ''.join(parts)
  return text

emails = pd.DataFrame()
emails['content'] = list(map(get_text_from_email, messages))

In [14]:
emails['Subject'] = list(map(lambda x: x.get(name="Subject"), messages))
emails['From'] = list(map(lambda x: x.get(name="From"), messages))
emails['To'] = list(map(lambda x: x.get(name="To"), messages))
emails['Date'] = list(map(lambda x: x.get(name="Date"), messages))
emails.head()

,content,Subject,From,To,Date
0,Here is our forecast\n\n,,phillip.allen@enron.com,tim.belden@enron.com,"Mon, 14 May 2001 16:39:00 -0700 (PDT)"
1,Traveling to have a business meeting takes the...,Re:,phillip.allen@enron.com,john.lavorato@enron.com,"Fri, 4 May 2001 13:51:00 -0700 (PDT)"
2,test successful. way to go!!!,Re: test,phillip.allen@enron.com,leah.arsdall@enron.com,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)"
3,"Randy,\n\n Can you send me a schedule of the s...",,phillip.allen@enron.com,randall.gay@enron.com,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)"
4,Let's shoot for Tuesday at 11:45.,Re: Hello,phillip.allen@enron.com,greg.piper@enron.com,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)"


In [75]:
def normalize_text(text):
  text = text.lower()
  
  # creating a space between a word and the punctuation following it to separate words
  # and compact repetition of punctuation
  # eg: "he is a boy.." => "he is a boy ."
  text = re.sub(r'([.,!?]+)', r" \1 ", text)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
  text = re.sub(r"[^a-zA-Z?.!,']+", " ", text)

  # Compact spaces
  text = re.sub(r'[" "]+', " ", text)

  # Remove forwarded messages
  text = text.split('forwarded by')[0]

  text = text.strip()

  return text

emails['content'] = list(map(normalize_text, emails['content']))

In [78]:
import numpy as np
emails['content'].replace('', np.nan, inplace=True)
emails.dropna(subset=['content'], inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_18152\4254048862.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  emails['content'].replace('', np.nan, inplace=True)


In [9]:
pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 11.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
nltk.download()

In [ ]:
import re
from tqdm import tqdm
import nltk
nltk.download('maxent_ne_chunker')
nltk.downnltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')



[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package ave

True

In [58]:
nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text)))

In [27]:
def remove_extensions(text):
    '''
    We removed attachments while extracting body but not the name of these attachments
    removing attachment_names based on what i encountered in subject and body
    '''
    ext_patterns = [r"\S+\.doc",r"\S+\.jpeg",r"\S+\.jpg",r"\S+\.gif",r"\S+\.csv",r"\S+\.ppt",r"\S+\.dat",r"\S+\.xml",r"\S+\.xls",r"\S+\.sql",r"\S+\.nsf",r"\S+\.jar",r"\S+\.bin"]
    pattern = '|'.join(ext_patterns)
    text = re.sub(pattern,'',text)
    return text

def remove_personal_name(text):
    '''
    Helper function to Filter out names using NER
    '''
    s = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text)))
    for ele in s:
        if isinstance(ele, nltk.Tree):
            if ele.label()=='PERSON':
                for word,pos_tag in ele:
                    try:     # words containing a special character will raise an error so handling it, these words weren't a name so we can safely skip it
                        val = re.sub(word,'',text)
                        text = val
                    except:
                        continue
    return text

def remove_subj_headers(x):
    '''
    To remove useless subject headers from data
    '''
    x = ' ' + x + ' '  # to make sure the regex work
    pattern = r'\s+(RE|re|Re|fw|FW|Fw)\s+'
    x = re.sub(pattern,'',x)
    return x.strip()


In [16]:
def transform_subject(s):
    '''
    Transform subject using some cleaning heuristics
    '''
    s = s.strip()

    # remove url and email-id's
    remove_url = r'(www|http)\S+'
    s = re.sub(remove_url,'',s)
    remove_email = r'\S+@\S+' 
    s = re.sub(remove_email,'',s)
    
    # remove attachment_names
    s = remove_extensions(s)

    # remove anything which is not a char
    remove_nonchars = r'[^A-Za-z_\']'
    s = re.sub(remove_nonchars,' ',s)

    # remove AM/PM as we have a lot of timestamps in subjects
    s = s.replace('AM','')
    s = s.replace('PM','')

    # remove personal names using named entity recognition from subject
    # s = remove_personal_name(s)  ---> wasn't optimal for subject as some words like 'Energy' and 'issues' were also recognized as Person entity

    # takes care of extra spaces 
    remove_space = r'\s+'
    s = re.sub(remove_space,' ',s)

    s = remove_subj_headers(s)
    return s

In [17]:
cleaned_subjects = []
for subj in tqdm(emails.Subject):
    cleaned_subjects.append(transform_subject(subj))

100%|██████████| 517401/517401 [00:16<00:00, 31168.77it/s]


In [18]:
emails.Subject = cleaned_subjects

In [19]:
# blanks
emails = emails[emails.content!='']
emails.drop_duplicates(subset=['content'], keep='first', inplace=True, ignore_index=True)
emails = emails.dropna(axis=0,subset=['content'])

emails = emails[~emails.From.isin(['noreply@ccomad3.uu.commissioner.com'])].reset_index(drop=True)  # drop noreply emails
emails = emails[~emails['content'].str.contains('Outlook Migration Team@ENRON')]    # drop outlook emails
emails = emails[~emails['content'].str.contains('Trade Counts and Volume')] # drop trade-logs emails


In [20]:
emails.shape

(248202, 5)

In [22]:
#Decontraction of text
def decontracted(phrase):
    """
    Returns decontracted phrases
    """
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [23]:
def remove_timestamps(text):
    '''
    Remove all types of 'text' data from timestamps
    '''
    text = text.replace('AM','')
    text = text.replace('PM','')
    text = text.replace('A.M.','')
    text = text.replace('P.M.','')
    text = text.replace('a.m.','')
    text = text.replace('p.m.','')
    text = re.sub(r"\bam\b",'',text)
    text = re.sub(r"\bpm\b",'',text)
    return text

In [87]:
def transform_body(text):
    '''
    In this function we have tried our best to clean the body as much as possible
    Comments tell what the code does.

    Overview :- 
    * since we have email threads, we will extract each email one by one based on delimiter/boundaries
    * we'll keep appending these emails in an array and return that
    * If there's just one mail, we have an array with just one element
    '''
    # remove metadata like forwarded by, original message text, etc.
    remove_forwarded = r'-{3,}.*Forwarded by.*'
    remove_orginal = r'-{3,}.*Original Message.*'
    remove_from = re.compile('From:.*',re.IGNORECASE)
    remove_sent= re.compile('Sent:.*',re.IGNORECASE)
    remove_to = re.compile('To:.*',re.IGNORECASE)
    remove_cc = re.compile('Cc:.*',re.IGNORECASE)
    remove_bcc = re.compile('Bcc:.*',re.IGNORECASE)
    remove_subject = re.compile('Subject:.*',re.IGNORECASE)
    # remove URL's
    remove_url = r'(www|http)\S+'     # https://stackoverflow.com/a/40823105
    remove_phone = '(\+\d{1,2}\s)?\d3?[\s.-]?\d{3}[\s.-]?\d{4}'   # ONLY US numbers for now --> https://stackoverflow.com/a/16699507

    #remove ANY emails
    remove_email = r'\S+@\S+'  # https://stackoverflow.com/a/64036475


    # Now before we do any preprocessing, we need to extraxt emails as we have 'email threads' not individual emails

    pattern = '(%s|%s)' % (remove_orginal,remove_forwarded)  # since FWD by & Original message headers are borders from one email to other.
    border_indices = [m.span() for m in re.finditer(pattern,text)][::-1]   # reverse it for chronological order i.e. if u want something like subject + prev-email to predict curr-prefix, that can also be done now
    temp = []
    if len(border_indices)>0:
        for idx,(start,end) in enumerate(border_indices):
            if idx==0:
                temp.append(text[end:])
            else:
                temp.append(text[end:prev_end])
            prev_end = end
        temp.append(text[:prev_end])  # sometimes there is some text even above the first header so storing that too

    def helper(text):
        pattern_list_1 = [remove_forwarded,remove_orginal,remove_from,remove_sent,remove_to,remove_cc,remove_bcc,remove_subject,remove_url,remove_phone,remove_email]

        for pattern in pattern_list_1:
            text = re.sub(pattern,'',text)

        # remove attachment_names
        text = remove_extensions(text)

        # remove any word with digit
        text = re.sub(r'\w*\d\w*', '', text)

        # remove any digit
        text = re.sub('\d','',text)

        # remove text between <>,()
        remove_tags = r'<.*?>'
        remove_brackets = r'\(.*?\)'
        remove_special_1 = r'\|-'  # remove raw backslash or '-'
        remove_colon = r'\b[\w]+:' # removes 'something:'


        pattern_list_2 = [remove_tags,remove_brackets,remove_special_1,remove_colon]
        for pattern in pattern_list_2:
            text = re.sub(pattern,'',text)
        

        # remove anything which is not a character,apostrophy ; remember to give a space on replacing with this
        remove_nonchars = r'[^A-Za-z\']'
        text = re.sub(remove_nonchars,' ',text)

        # remove AM/PM as we have a lot of timestamps in emails
        text = remove_timestamps(text)

        # remove personal names using named entity recognition
        text = remove_personal_name(text)

        # takes care of \t & \n ; remember to give a space on replacing with this
        remove_space = r'\s+'
        text = re.sub(remove_space,' ',text)

        # take care of apostrophies
        text = decontracted(text)

        # remove other junk
        text = text.replace("IMAGE",'')
        text = re.sub(r"\bth\b",'',text)

        return text.strip()

    if len(temp) > 0:
        # extract several mails from a thread
        for i in range(len(temp)):
            temp[i] = helper(temp[i])
    else:
        # just store that single mail in array
        temp.append(helper(text))
    return temp


<>:22: SyntaxWarning: invalid escape sequence '\+'
<>:55: SyntaxWarning: invalid escape sequence '\d'
<>:22: SyntaxWarning: invalid escape sequence '\+'
<>:55: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Admin\AppData\Local\Temp\ipykernel_9688\1626234050.py:22: SyntaxWarning: invalid escape sequence '\+'
  remove_phone = '(\+\d{1,2}\s)?\d3?[\s.-]?\d{3}[\s.-]?\d{4}'   # ONLY US numbers for now --> https://stackoverflow.com/a/16699507
C:\Users\Admin\AppData\Local\Temp\ipykernel_9688\1626234050.py:55: SyntaxWarning: invalid escape sequence '\d'
  text = re.sub('\d','',text)


In [88]:
# sample cleaned body
idx = 6000
print("Email corpus:",end='\n\n')
print(emails.iloc[idx].content[:100]) 
print('\n\n')
print("After Transformation...")
text = emails.iloc[idx].content
transform_body(text)


Email corpus:

  
Redskins to stick with Johnson at quarterback 
 Sept. 19, 2000
By Len Pasquarelli
SportsLine.com 



After Transformation...


['I need for you to find out how much the balance on your tv is and let me know',
 "Redskins to stick with at quarterback Sept By com Senior Writer Despite the continuing struggles of the offense and costly fourth quarter interceptions by the Washington Redskins will stick with their starting quarterback for Sunday night is game against the New York com has confirmed If it was just with then we would look at it but that is not the case coach said But it goes beyond just the quarterback position It is more than that We are not playing well on offense in general Redskins quarterback will remain the team is starter against the unbeaten In the wake of the Redskins' loss to the Dallas Cowboys on Monday night there were rumblings Washington might switch to backup for the key matchup against the undefeated The Washington offense one of the league is most diverse in when it ranked No overall and second in scoring has been sporadic at best in three games A seventh year veteran who is in the fin

In [98]:
cleaned_body_all = []
final_cleaned_subj = []
emails_sample = emails
for text,subj in tqdm(zip(emails_sample.content,emails_sample.Subject),total = len(emails_sample.content)):
    result = transform_body(text)
    cleaned_body_all.extend(result)

  6%|▌         | 13820/248202 [4:34:57<77:43:14,  1.19s/it]     


KeyboardInterrupt: 

In [99]:
import pickle
dataset = [cleaned_body_all]
with open('final_body_all.pickle', 'wb') as file:
    pickle.dump(dataset, file)
